<a href="https://colab.research.google.com/github/manigamer22/weatherwise-subash-kovvuri/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

📄 **Full Assignment Specification**  
See [`ASSIGNMENT.md`](ASSIGNMENT.md) or check the LMS for full details.

📝 **Quick Refresher**  
A one-page summary is available in [`resources/assignment-summary.md`](resources/assignment-summary.md).

---

🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.




## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [ ]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai
!pip install pyinputplus

# ✅ Import after installing (if needed)
from fetch_my_weather import get_weather
from hands_on_ai import prompt_logger

#api: key: dc54ae66a5dcb61c0e4816e5be860da2   <-- use this key to get all the data for the weather!

# Go to https://openweathermap.org/api to got my own API key.

import os

os.environ['HANDS_ON_AI_SERVER'] = "https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API key}"
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')


## 📦 Setup and Configuration
Import required packages and setup environment.

In [13]:
# ✅ STEP 1: Install Required Packages
!pip install fetch-my-weather --quiet
!pip install hands-on-ai --quiet
!pip install pyinputplus --quiet
!pip install matplotlib --quiet

# ✅ STEP 2: Imports
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.8/459.8 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 82.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently ta

## 🌤️ Weather Data Functions

In [ ]:
# ✅ STEP 3: Weather API Setup
API_KEY = 'dc54ae66a5dcb61c0e4816e5be860da2'

# ✅ STEP 4: Weather Data Fetching Function
def get_weather_data(city):
    """
    Fetches weather data for the specified city using OpenWeatherMap API.
    Returns JSON response or None if error.
    """
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: Unable to fetch weather data ({response.status_code})")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

## 📊 Visualisation Functions

In [ ]:
def show_temperature_chart(temps, days):
    """
    Creates a 3D-style colorful temperature line chart.
    """
    plt.figure(figsize=(10, 6))
    plt.plot(days, temps, marker='o', linestyle='-', linewidth=2, color='orange')
    plt.fill_between(days, temps, color='orange', alpha=0.3)
    plt.title("🌡️ Temperature Forecast", fontsize=16)
    plt.xlabel("Day", fontsize=12)
    plt.ylabel("Temperature (°C)", fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.style.use('seaborn-dark-palette')
    plt.show()


def show_precipitation_chart(precip, times):
    """
    Creates a bar chart for precipitation with a dark theme.
    """
    plt.figure(figsize=(10, 6))
    bars = plt.bar(times, precip, color='dodgerblue', edgecolor='white')
    plt.title("🌧️ Precipitation Forecast", fontsize=16)
    plt.xlabel("Time", fontsize=12)
    plt.ylabel("Precipitation (%)", fontsize=12)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.style.use('dark_background')
    plt.show()


## 🤖 Natural Language Processing

In [ ]:
def parse_weather_question(query):
    """
    Parses the user's weather question and identifies intent.
    """
    query = query.lower()
    if "umbrella" in query:
        return "umbrella"
    elif "temperature" in query:
        return "temperature"
    elif "rain" in query:
        return "rain"
    return "unknown"


def generate_weather_response(query_type, weather_data):
    """
    Generates a natural response based on weather query type.
    """
    if query_type == "umbrella":
        condition = weather_data["weather"][0]["main"].lower()
        return "Yes, bring an umbrella." if "rain" in condition else "No umbrella needed."
    elif query_type == "temperature":
        temp = weather_data["main"]["temp"]
        return f"The current temperature is {temp}°C."
    elif query_type == "rain":
        condition = weather_data["weather"][0]["description"]
        return f"Weather forecast: {condition}."
    else:
        return "I didn't understand that question."

## 🧭 User Interface

In [8]:
def menu():
    """
    Main menu for user interaction.
    """
    while True:
        choice = pyip.inputMenu(['Get Weather', 'Ask a Question', 'Show Chart', 'Exit'], numbered=True)

        if choice == 'Get Weather':
            city = input("Enter city: ")
            data = get_weather_data(city)
            if data:
                print(f"{city.title()} Weather: {data['weather'][0]['description']}, Temp: {data['main']['temp']}°C")
            else:
                print("City not found or API error.")

        elif choice == 'Ask a Question':
            city = input("Enter city: ")
            data = get_weather_data(city)
            if not data:
                print("City not found.")
                continue
            query = input("Ask a weather question (e.g., Do I need an umbrella?): ")
            q_type = parse_weather_question(query)
            response = generate_weather_response(q_type, data)
            print(response)

        elif choice == 'Show Chart':
            days = ["Mon", "Tue", "Wed", "Thu", "Fri"]
            temps = [21, 23, 22, 24, 25]  # Replace with actual forecast later if extended
            show_temperature_chart(temps, days)
        else:
            print("Goodbye!")
            break

## 🧩 Main Application Logic

In [ ]:
# ✅ Install required packages (run this once)
!pip install pyinputplus --quiet
!pip install matplotlib --quiet

# ✅ Import packages
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip

# ✅ API Key Setup (replace with your own key)
API_KEY = "dc54ae66a5dcb61c0e4816e5be860da2"

# 🌤️ Weather Data Function
def get_weather(city):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# 🤖 Simple NLP Parser and Response Generator
def parse_weather_question(query):
    query = query.lower()
    if "umbrella" in query:
        return "umbrella"
    elif "temperature" in query:
        return "temperature"
    return "unknown"

def generate_weather_response(query_type, weather_data):
    if query_type == "umbrella":
        condition = weather_data["weather"][0]["main"].lower()
        if "rain" in condition or "drizzle" in condition:
            return "Yes, you should take an umbrella."
        else:
            return "No umbrella needed today."
    elif query_type == "temperature":
        temp = weather_data["main"]["temp"]
        return f"The temperature is {temp}°C."
    else:
        return "Sorry, I didn't understand that."

# 📊 Temperature Chart (dummy example)
def show_temperature_chart(temps, days):
    plt.plot(days, temps, marker='o')
    plt.title("Temperature Forecast")
    plt.xlabel("Day")
    plt.ylabel("Temperature (°C)")
    plt.grid(True)
    plt.show()

# 🧭 Menu Function
def menu():
    while True:
        choice = pyip.inputMenu(['Get Weather', 'Ask a Question', 'Show Chart', 'Exit'], numbered=True)

        if choice == 'Get Weather':
            city = input("Enter city: ")
            data = get_weather(city)
            if data:
                print(f"{city} Weather: {data['weather'][0]['description']}, Temp: {data['main']['temp']}°C")
            else:
                print("City not found or API error.")

        elif choice == 'Ask a Question':
            city = input("Enter city: ")
            data = get_weather(city)
            if not data:
                print("City not found.")
                continue
            query = input("Ask a weather question: ")
            q_type = parse_weather_question(query)
            response = generate_weather_response(q_type, data)
            print(response)

        elif choice == 'Show Chart':
            # Dummy data - you can later fetch real forecast
            days = ["Mon", "Tue", "Wed", "Thu", "Fri"]
            temps = [22, 24, 21, 23, 25]
            show_temperature_chart(temps, days)

        else:
            print("Goodbye!")
            break

# ▶️ Run the app
menu()

## 🧪 Testing and Examples

In [16]:
# ✅ STEP 1: Install Required Packages
# Installing external libraries needed for weather fetching, AI chat, user input validation, and plotting
!pip install fetch-my-weather --quiet
!pip install hands-on-ai --quiet
!pip install pyinputplus --quiet
!pip install matplotlib --quiet

# ✅ STEP 2: Imports
# Importing necessary modules for HTTP requests, plotting, user input validation, and AI chat interaction
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response

# ✅ STEP 3: Weather API Setup
# OpenWeatherMap API key for authentication with the weather data service
API_KEY = 'dc54ae66a5dcb61c0e4816e5be860da2'

# ✅ STEP 4: Weather Data Fetching Function
def get_weather_data(city):
    """
    Fetches current weather data for the specified city from OpenWeatherMap API.

    Parameters:
        city (str): The name of the city to fetch weather for.

    Returns:
        dict or None: JSON response containing weather data if successful, else None.
    """
    # Construct the API request URL with city, API key, and metric units
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    try:
        response = requests.get(url)
        # Check if the request was successful
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: Unable to fetch weather data ({response.status_code})")
            return None
    except requests.exceptions.RequestException as e:
        # Handle exceptions like connection errors
        print(f"Request failed: {e}")
        return None

# ✅ STEP 5: Visualisation Functions

def show_temperature_chart(temps, days):
    """
    Creates a 3D-style colorful temperature line chart showing temperature trends over days.

    Parameters:
        temps (list of int/float): Temperature values to plot.
        days (list of str): Corresponding days for the temperature values.
    """
    # Use seaborn-dark style for better visualization aesthetics
    plt.style.use('seaborn-dark')
    plt.figure(figsize=(10, 6))
    # Plot line chart with orange color and markers for each day
    plt.plot(days, temps, marker='o', linestyle='-', linewidth=2, color='orange')
    # Fill the area under the line with semi-transparent orange color
    plt.fill_between(days, temps, color='orange', alpha=0.3)
    plt.title("Temperature Forecast", fontsize=16)
    plt.xlabel("Day", fontsize=12)
    plt.ylabel("Temperature (°C)", fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.show()


def show_precipitation_chart(precip, times):
    """
    Creates a bar chart for precipitation percentages over time with a dark theme.

    Parameters:
        precip (list of int/float): Precipitation values (percentage).
        times (list of str): Corresponding times or labels for the precipitation values.
    """
    plt.style.use('dark_background')
    plt.figure(figsize=(10, 6))
    # Bar chart with dodgerblue bars and white edges for contrast
    bars = plt.bar(times, precip, color='dodgerblue', edgecolor='white')
    plt.title("Precipitation Forecast", fontsize=16)
    plt.xlabel("Time", fontsize=12)
    plt.ylabel("Precipitation (%)", fontsize=12)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.show()

# ✅ STEP 6: Natural Language Processing Functions

def parse_weather_question(query):
    """
    Parses the user's weather-related question and identifies the type of intent.

    Parameters:
        query (str): The user input question.

    Returns:
        str: The identified query type ('umbrella', 'temperature', 'rain', or 'unknown').
    """
    query = query.lower()
    # Check keywords to determine user intent
    if "umbrella" in query:
        return "umbrella"
    elif "temperature" in query:
        return "temperature"
    elif "rain" in query:
        return "rain"
    return "unknown"

def generate_weather_response(query_type, weather_data):
    """
    Generates a natural language response based on the query type and weather data.

    Parameters:
        query_type (str): The type of question parsed.
        weather_data (dict): JSON weather data from API.

    Returns:
        str: A response string tailored to the user's question.
    """
    if query_type == "umbrella":
        # Check weather condition for rain to decide umbrella need
        condition = weather_data["weather"][0]["main"].lower()
        return "Yes, bring an umbrella." if "rain" in condition else "No umbrella needed."
    elif query_type == "temperature":
        # Provide current temperature
        temp = weather_data["main"]["temp"]
        return f"The current temperature is {temp}°C."
    elif query_type == "rain":
        # Describe rain or other precipitation
        condition = weather_data["weather"][0]["description"]
        return f"Weather forecast: {condition}."
    else:
        return "I didn't understand that question."

# ✅ STEP 7: User Interface Menu

def menu():
    """
    Main interactive menu loop for the WeatherWise application.
    Allows user to fetch weather, ask questions, view charts, or exit.
    """
    while True:
        # Display a numbered menu for user selection
        choice = pyip.inputMenu(['Get Weather', 'Ask a Question', 'Show Chart', 'Exit'], numbered=True)

        if choice == 'Get Weather':
            # Prompt user for city and fetch current weather data
            city = input("Enter city: ")
            data = get_weather_data(city)
            if data:
                # Display summary of weather and temperature
                print(f"{city.title()} Weather: {data['weather'][0]['description']}, Temp: {data['main']['temp']}°C")
            else:
                print("City not found or API error.")

        elif choice == 'Ask a Question':
            # Allow user to ask natural language questions about weather
            city = input("Enter city: ")
            data = get_weather_data(city)
            if not data:
                print("City not found.")
                continue
            query = input("Ask a weather question (e.g., Do I need an umbrella?): ")
            # Parse question intent and generate appropriate response
            q_type = parse_weather_question(query)
            response = generate_weather_response(q_type, data)
            print(response)

        elif choice == 'Show Chart':
            # Show a sample temperature forecast chart (placeholder data)
            days = ["Mon", "Tue", "Wed", "Thu", "Fri"]
            temps = [21, 23, 22, 24, 25]  # TODO: Replace with real forecast data if available
            show_temperature_chart(temps, days)
        else:
            # Exit the program loop
            print("Goodbye!")
            break

# ✅ STEP 8: Run the Application
# Start the interactive WeatherWise menu application
menu()


Please select one of the following:
1. Get Weather
2. Ask a Question
3. Show Chart
4. Exit
sydney
'sydney' is not a valid choice.
Please select one of the following:
1. Get Weather
2. Ask a Question
3. Show Chart
4. Exit
1
Enter city: sydney
Sydney Weather: overcast clouds, Temp: 16.45°C
Please select one of the following:
1. Get Weather
2. Ask a Question
3. Show Chart
4. Exit
3


OSError: 'seaborn-dark' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.